In [14]:
import numpy as np
import xarray as xr
import time 
import os
from netCDF4 import Dataset
from pyproj import CRS
import rasterio
import rioxarray
from rasterio.enums import Resampling
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
from sklearn.model_selection import GroupKFold, GridSearchCV


In [2]:
sinmod_unstandardized = xr.open_dataset('/cluster/home/maikents/sinmod_features_masters/sinmod_bottom_features_emod_grid.nc')

In [3]:
print(sinmod_unstandardized )

<xarray.Dataset> Size: 14GB
Dimensions:                                (stat: 5, x: 8319, y: 9668)
Coordinates:
  * stat                                   (stat) <U20 400B '10th_percentile'...
  * x                                      (x) float64 67kB -6.546e+04 ... 6....
  * y                                      (y) float64 77kB 7.539e+06 ... 6.6...
Data variables:
    spatial_ref                            int64 8B ...
    bottom_temperature_sundahl_features    (stat, y, x) float32 2GB ...
    bottom_temperature_features            (stat, y, x) float32 2GB ...
    bottom_salinity_features               (stat, y, x) float32 2GB ...
    bottom_current_features                (stat, y, x) float32 2GB ...
    bottom_statistical_northness_features  (stat, y, x) float64 3GB ...
    bottom_statistical_eastness_features   (stat, y, x) float64 3GB ...
    current_aspect_angle                   (y, x) float64 643MB ...


In [4]:
temp = sinmod_unstandardized['bottom_temperature_features']

In [5]:
print(temp)

<xarray.DataArray 'bottom_temperature_features' (stat: 5, y: 9668, x: 8319)> Size: 2GB
[402140460 values with dtype=float32]
Coordinates:
  * stat     (stat) <U20 400B '10th_percentile' ... 'min_(mean_march_may)'
  * x        (x) float64 67kB -6.546e+04 -6.537e+04 ... 6.914e+05 6.915e+05
  * y        (y) float64 77kB 7.539e+06 7.539e+06 ... 6.66e+06 6.659e+06
Attributes:
    grid_mapping:  spatial_ref


In [6]:
temp = temp.sel(stat='mean')

In [11]:
combined_df_nans = pd.read_parquet("/cluster/home/maikents/sinmod_features_masters/dataframes_pipeline_5/total_df_with_nans_and_labels.parquet")

x_coords = combined_df_nans['x']
y_coords = combined_df_nans['y']

In [15]:
res_x = (x_coords.max() - x_coords.min()) / temp.shape[1]
res_y = (y_coords.max() - y_coords.min()) / temp.shape[0]

transform = from_origin(x_coords.min(), y_coords.max(), res_x, res_y)

#Save to GeoTIFF with correct CRS
with rasterio.open(
    "/cluster/home/maikents/sinmod_features_masters/temperature_plotting.tif",
    "w",
    driver="GTiff",
    height=temp.shape[0],
    width=temp.shape[1],
    count=1,
    dtype="float32",
    crs="EPSG:25833",  #EMODnet UTM Zone 33N / ETRS89
    transform=transform,
) as dst:
    dst.write(temp.astype("float32"), 1)

In [17]:

with rasterio.open("/cluster/home/maikents/sinmod_features_masters/temperature_plotting.tif") as src:
    data = src.read(1)

    flipped_data = np.flipud(data)

    meta = src.meta.copy()

    with rasterio.open('/cluster/home/maikents/sinmod_features_masters/temperature_flipped_vertically_finished.tif', 'w', **meta) as dst:
        dst.write(flipped_data, 1)

print("Vertical flip complete.")


Vertical flip complete.


In [18]:

with rasterio.open('/cluster/home/maikents/sinmod_features_masters/temperature_flipped_vertically_finished.tif') as src:
    data = src.read(1)
    profile = src.profile

data_clean = np.where(np.isnan(data), -9999, data)

profile.update(nodata=-9999)

with rasterio.open('/cluster/home/maikents/sinmod_features_masters/temp_final.tif', "w", **profile) as dst:
    dst.write(data_clean, 1)
